# Datasets

การสร้าง dataset เป็นการจัดเก็บ LLMtestCase หรือ Golden หลายๆตัวรวมกันเพื่อ evaluate ทีเดียว

In [9]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

In [11]:
azure_openai.generate("hello")

'Hello! How can I help you today?'

### Create An Evaluation Dataset

เพิ่มแบบ Test cases

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

first_test_case = LLMTestCase(input="...", actual_output="...")
second_test_case = LLMTestCase(input="...", actual_output="...")

test_cases = [first_test_case, second_test_case]

dataset = EvaluationDataset(test_cases=test_cases)

เพิ่มแบบ golden หมายความว่าชุดข้อมูลนั้นมีเพียงอินพุต ไม่มีเอาพุตที่เตรียมไว้ล่วงหน้า

In [ ]:
from deepeval.dataset import EvaluationDataset, Golden

first_golden = Golden(input="...")
second_golden = Golden(input="...")

goldens = [first_golden, second_golden]

dataset = EvaluationDataset(goldens=goldens)

### Generate An Evaluation Dataset

นอกจากนี้ยังสามารถใช้วิธีการ synthetic เพื่อสร้างข้อมูลจากชุด document ซึ่งมีประโยชน์หากไม่ได้เตรียมชุดข้อมูลประเมินไว้

In [ ]:
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer


dataset = EvaluationDataset()
synthesizer = Synthesizer(model=azure_openai)
dataset.generate_goldens_from_docs(
    synthesizer=synthesizer,
    document_paths=['gpt4o.txt'],
    max_goldens_per_document=2
)

# Load an Existing Dataset

บางครั้งเราได้มีการเตรียมชุดข้อมูลไว้แล้ว ที่ประกอบไปด้วย input, output, expect_output, context, retrieval_context อยู่ในรูปแบบ json หรือ csv แล้วสามารถแปลงเป็น dataset ได้ดังนี้

### From JSON

In [ ]:
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.add_test_cases_from_json_file(
    # file_path is the absolute path to you .json file
    file_path="example.json",
    input_key_name="query",
    actual_output_key_name="actual_output",
    expected_output_key_name="expected_output",
    context_key_name="context",
    retrieval_context_key_name="retrieval_context",
)

### From CSV

In [ ]:
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.add_test_cases_from_csv_file(
    # file_path is the absolute path to you .csv file
    file_path="example.csv",
    input_col_name="query",
    actual_output_col_name="actual_output",
    expected_output_col_name="expected_output",
    context_col_name="context",
    context_col_delimiter= ";",
    retrieval_context_col_name="retrieval_context",
    retrieval_context_col_delimiter= ";"
)